In [3]:
%matplotlib inline

# Sentence Classification with LSTMs

**Course:** CMSC 389A - Practical Deep Learning 

**Author:** Sujith Vishwajith   

**Task:** Up until now we have mainly been discussing non-sequential data of a fixed size. For example with our dense feed forward neural networks, we always passed in some feature vector of size _N_, and for convolutional neural networks we always passed in images of the same size. We will now move towards building models that can handle inputs where we care about some time based order (e.g. word order, stock data) and an input of an unspecified size (e.g. a sentence). Specificaly, for this task we will classify whether movie reviews on IMDB are positive or negative in sentiment. Based off of previous classes, we know that this is a binary classificaiton problem.

This notebook is meant to supplement this weeks lecture on LSTM's and give a reference for Practical 4.
 
**Packages**  
Lets import the following required packages.

In [23]:
import numpy as np
from keras.datasets import imdb
from keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.preprocessing import sequence

# Fixed seed for reproducibility
kSEED = 5
np.random.seed(kSEED)

## Load Data

We'll be using the IMDB dataset (Large Movie Review Dataset) which is easily accessible through Keras. The dataset contains 25,000 labeled movie reviews for training and another 25,000 for testing.

We will pass in a value of 5000 to the argument num_words which basically gets rid of all words that aren't in the top 5000 most frequent word list. This is a type of preprocessing which is useful for getting rid of rare and never seen words that mess up classification.

In [7]:
kTOP = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=kTOP)

17465344/17464789 [==============================] - 3s 0us/step


We also want to cut off sentences greater than a certain length and pad sentences under a certain length so all inputs are of size _N_ when training our model to improve performance. In our case, we will use a fixed sentence length of 500.

For example, let's say we have two sentences "i am a boy" and "i am a tall boy in high school" and our max length is 5. The strings would now become "i am a boy 0" and "i am a tall boy" respectively. Noticed how the first string was padded with the character 0. The model will learn that that the specical character (in this example it is 0) means nothing and that it should ignore it.

In [8]:
kMAXLEN = 500
X_train = sequence.pad_sequences(X_train, maxlen=kMAXLEN)
X_test = sequence.pad_sequences(X_test, maxlen=kMAXLEN)

## Building the Model (LSTM only)

In this section we will build our first LSTM model. Since this is a binary classification task, we will use the binary_crossentropy loss function. The goal of the model is to output a value between 0 and 1 for the respective classes. Our model will have a single LSTM layer with neurons (memory units). Since LSTMS sometimes have a problem with overfitting, we will add two dropout layers to help prevent that. We will only train the model for 1 epoch (not a good practice) for quick testing.

### Embedding Layer

If we remember in class, we discussed a concept called word embeddings. This refers to a mapping between a word to a vector representation of that word. The benefits of doing this is that the word is now numeric; meaning we can do math between different words. For example, we can take the cosine similarity between the word embedding vectors of two words and the output would be how similar the words are in meaning. 

In this step, we have two options. We could either add an embedding layer which trains and learns the word embeddings while training the model or we could use a pretrained word embedding model and convert the sequence of words into a sequence of vectors which we could then pass in. Due to the small size of our dataset, we will use the former approach and learn embeddings while training. Keras makes this simple with an _Embedding_ layer. Note that for larger datasets and more complex problems, it is probably better and faster to pass in pretrained embeddings. 

For the size of our embeddings, we will use a vector of size 50. This is because our vocabulary isn't very large and so we won't have to capture a ton of meaning in a vector as these are all movie related. However, the more complex your text based data is, the larger size embedding you will need. For example, most pretrained models have embedding vectors of size 300.

In [18]:
kVECTORLEN = 50

model = Sequential()
model.add(Embedding(kTOP, kVECTORLEN, input_length=kMAXLEN))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 500, 50)           250000    
_________________________________________________________________
dropout_7 (Dropout)          (None, 500, 50)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dropout_8 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 310,501
Trainable params: 310,501
Non-trainable params: 0
_________________________________________________________________
None
Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 242s 10ms/step - loss: 0

We can now evaluate our model's performance.

In [19]:
scores = model.evaluate(X_test, y_test, verbose=0)
accuracy = (scores[1]*100)

print("Accuracy: {:.2f}%".format(scores[1]*100))

Accuracy: 84.40%


## Building the Model (LSTM + CNN)

If we remember from previous concepts, CNNs are excellent at learning spatial structure from data. We can take advantage of this and actually learn some structure from our sequential data that we could then pass into our LSTM layer. One way to think about this is that by adding a convolutional layer before the LSTM layer, we are passing in sequences of chunks rather than a sequence of words. The goal is that the chunks are more informative concepts than individual words.

For example rather than just passing in the sequence "I love to run in the morning" to an LSTM, we can use a convolutional layer which will learn to pass in something like \["I love to", "run in the morning"\] to the LSTM. 

In [24]:
model = Sequential()
model.add(Embedding(kTOP, kVECTORLEN, input_length=kMAXLEN))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=64)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 500, 50)           250000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 32)           4832      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 250, 32)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_10 (Dropout)         (None, 100)               0         
___________________________________________________________

We can now evaluate our new model's performance.

In [26]:
scores = model.evaluate(X_test, y_test, verbose=0)
accuracy = (scores[1]*100)

print("Accuracy: {:.2f}%".format(scores[1]*100))

Accuracy: 88.25%


Observe that adding the convolutional layers dramatically sped up the training time and improved the performance.